<a href="https://colab.research.google.com/github/Vadim456456/Calculator/blob/main/SQL_PostgreSQL_Python_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задачи по SQL

Входные данные. Даны 3 таблицы dep, emp, fin

dep: id, name

emp: id, name, dep_id, age, salary

fin: month, emp_id, salary

dep:
1, Бухгалтерия

2, Кадры

emp:

1, Иванов, 1,45,100

2, Петров, 1,34,86

3, Петров, 1,18,75

4, Петров, 1,56,92

5, Сидоров, 1,28,101

6, Сидоров, 1,37,79

7, Сидоров, 2,31,55

8, Сидоров, 2,30,67

9, Федотов, 2,18,43

In [50]:
import sqlite3

In [51]:
conn = sqlite3.connect(':memory:') #Создаю соединение БД
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE dep (
    id INTEGER PRIMARY KEY,
    name TEXT
)
''')
#Создал таблицу dep
cursor.executemany('''
INSERT INTO dep (id, name) VALUES (?, ?)
''', [
    (1, 'Бухгалтерия'),
    (2, 'Кадры')
]) #Заполнил таблицу dep

conn.commit()

cursor.execute('SELECT * FROM dep')
results_dep = cursor.fetchall()

for row_dep in results_dep:
    print(row_dep)

(1, 'Бухгалтерия')
(2, 'Кадры')


In [ ]:
cursor.execute('''
CREATE TABLE emp (
    id INTEGER PRIMARY KEY,
    name TEXT,
    dep_id INTEGER,
    age INTEGER,
    salary REAL
)
''') #Создал таблицу emp

cursor.executemany('''
INSERT INTO emp (id, name, dep_id, age, salary) VALUES (?, ?, ?, ?, ?)
''', [
    (1, 'Иванов', 1, 45, 100),
    (2, 'Петров', 1, 34, 86),
    (3, 'Петров', 1, 18, 75),
    (4, 'Петров', 1, 56, 92),
    (5, 'Сидоров', 1, 28, 101),
    (6, 'Сидоров', 1, 37, 79),
    (7, 'Сидоров', 2, 31, 55),
    (8, 'Сидоров', 2, 30, 67),
    (9, 'Федотов', 2, 18, 43)
]) #Заполнил таблицу emp

conn.commit()

cursor.execute('SELECT * FROM emp')
results_emp = cursor.fetchall()

for row_emp in results_emp:
    print(row_emp)

(1, 'Иванов', 1, 45, 100.0)
(2, 'Петров', 1, 34, 86.0)
(3, 'Петров', 1, 18, 75.0)
(4, 'Петров', 1, 56, 92.0)
(5, 'Сидоров', 1, 28, 101.0)
(6, 'Сидоров', 1, 37, 79.0)
(7, 'Сидоров', 2, 31, 55.0)
(8, 'Сидоров', 2, 30, 67.0)
(9, 'Федотов', 2, 18, 43.0)


In [ ]:
cursor.execute('''
CREATE TABLE fin (
    month INTEGER,
    emp_id INTEGER,
    salary INTEGER
)
''') # Создал таблицу fin

cursor.executemany('''
INSERT INTO fin (month, emp_id, salary) VALUES (?, ?, ?)
''', [
    (1, 1, 50),
    (2, 1, 50),
    (3, 1, 50),
    (1, 2, 60),
    (2, 2, 60),
    (3, 2, 60)
])

conn.commit()

cursor.execute('SELECT * FROM fin')
results_fin = cursor.fetchall()

for row_fin in results_fin:
    print(row_fin)

(1, 1, 50)
(2, 1, 50)
(3, 1, 50)
(1, 2, 60)
(2, 2, 60)
(3, 2, 60)


In [ ]:

cursor.close()
conn.close()

--1 Задание. Даны таблицы emp и dep. Необходимо посчитать количество однофамильцев в отделах.

-- Итоговый результат, который должен получится:

Бухгалтерия, 5

Кадры, 2

In [ ]:
query = '''
WITH counted_emp AS (
    SELECT
        d.name AS department_name,
        e.name AS employee_name,
        COUNT(*) OVER (PARTITION BY e.name, d.id) AS employee_count
    FROM emp e
    JOIN dep d ON e.dep_id = d.id
)
SELECT department_name,
    SUM(CASE WHEN employee_count > 1 THEN 1 ELSE 0 END) AS onefamilies_count
FROM counted_emp
GROUP BY department_name;
'''

cursor.execute(query)
results = cursor.fetchall()

for result_counted in results:
    print(f"{result_counted[0]}, {result_counted[1]}")

Бухгалтерия, 5
Кадры, 2


--2 Задание. Дана таблица fin с заполненными полями month, emp_id, salary. Необходимо посчитать нарастающий итог по каждому сотруднику.

--Итоговый результат отобразить в поле sum_salary

fin: month, emp_id, salary, sum_salary

1,1,50,50

2,1,50,100

3,1,50,150

1,2,60,60

2,2,60,120

3,2,60,180

In [ ]:
query_fin = '''
SELECT
    month,
    emp_id,
    salary,
    SUM(salary) OVER (PARTITION BY emp_id ORDER BY month) AS sum_salary
FROM
    fin
'''

cursor.execute(query_fin)
results_fin = cursor.fetchall()

for result_fin in results_fin:
    print(result_fin)

(1, 1, 50, 50)
(2, 1, 50, 100)
(3, 1, 50, 150)
(1, 2, 60, 60)
(2, 2, 60, 120)
(3, 2, 60, 180)


-- 3 Задание. Имеется таблица dep2 в которой появились полные дубли записей.

Необходимо восстановление уникальности записей в таблице и привести ее к виду dep1

dep1

1, Бухгалтерия

2, Кадры

dep2

1,Бухгалтерия 1

1,Бухгалтерия 2

1,Бухгалтерия 3

1,Бухгалтерия 4

1,Бухгалтерия 5

2, Кадры 1

2, Кадры 2

2, Кадры 3

In [ ]:
cursor.execute('''
CREATE TABLE dep2 AS
SELECT *
FROM dep
''')

conn.commit()

cursor.execute('SELECT * FROM dep2')
results_dep2 = cursor.fetchall()

for row_dep2 in results_dep2:
    print(row_dep2)

#Копировали таблицу dep и с помощью её сделали dep2

(1, 'Бухгалтерия')
(2, 'Кадры')


In [ ]:
unique_table = '''
SELECT *
FROM dep
UNION
SELECT *
FROM dep2 '''

cursor.execute(unique_table)
results_dep2 = cursor.fetchall()

for result_dep2 in results_dep2:
    print(f"{result_dep2[0]}, {result_dep2[1]}")

1, Бухгалтерия
2, Кадры


--4 Задание. Оптимизация запросов в SQL

Имеется базы, в которой есть столбец с датой и временем - они указаны по Гринвичу (-3 от МСК)

На столбец start_date есть индекс.

Как оптимизировать следующий запрос, если дата и время на выходе нам нужны уже по МСК:

declare @start as datetime= "2000-03-01 00:00:00";

declare @end as datetime = "2000-03-31 23:59:59";

select name, start_date as time_utc, dateadd(hour, 3, start_date) as time_msk
from DQ.DB.employers
where dateadd(hour, 3, start_date) between @start and @end

In [53]:
conn = sqlite3.connect(':time_change_to_msk:')
cursor = conn.cursor()

In [60]:
cursor.execute('''
SELECT
    '2000-03-01 00:00:00' AS start_time_gmt,
    datetime('2000-03-01 00:00:00', '+3 hours') AS start_time_msk,
    '2000-03-31 23:59:59' AS end_time_gmt,
    datetime('2000-03-31 23:59:59', '+3 hours') AS end_time_msk;
    ''')

result = cursor.fetchall()

for row in result:
    print(f"Start Time GMT: {row[0]}, Start Time MSK: {row[1]}") #Отображение по Гринвичу
    print(f"End Time GMT: {row[2]}, End Time MSK: {row[3]}") #Отображение по Московскому времени

Start Time GMT: 2000-03-01 00:00:00, Start Time MSK: 2000-03-01 03:00:00
End Time GMT: 2000-03-31 23:59:59, End Time MSK: 2000-04-01 02:59:59


-- Python

--5 Задание. Дана ведомость в которой два столбца - отдел и сотрудник.

Необходимо подсчитать количество уникальных сотрудников в каждом отделе. Результат отразить в поле Cnt

df: A,B

Отдел Сотрудник Cnt

1 1 3

1 2 3

2 3 1

3 6 1

1 7 3

3 6 1

In [8]:
import pandas as pd

In [10]:
data = {
    'A': [1, 1, 2, 3, 1, 3],
    'B': [1, 2, 3, 6, 7, 6]
}

df = pd.DataFrame(data, columns=['A', 'B'])
unique_counts = df.groupby('A')['B'].nunique().reset_index(name='Cnt')
df = df.merge(unique_counts, on='A', how='left')

print(df.to_string(index=False)) #Убираем индекс

 A  B  Cnt
 1  1    3
 1  2    3
 2  3    1
 3  6    1
 1  7    3
 3  6    1


-- 6 Задание. Дан df содержащий количество авторизаций сотрудников по неделям.
Необходимо посчитать количество авторизаций по каждому сотруднику за каждую неделю и суммарное количество с начала периода.

Результат отобразить в полях cnt и sum

df= pd.DataFrame( {'w':[1,1,2,2,2,3,3,3,4,4,4],

'user':['User2', 'User3', 'User1', 'User1', 'User2', 'User2', 'User2', 'User3', 'User1', 'User1', 'User3'],

'c':[7,2,3,1,2,5,4,6,9,8,7]})

Входные данные:

w=week user cnt

1 User2 7

1 User3 2

2 User1 3

2 User1 1

2 User2 2

3 User2 5

3 User2 4

3 User3 6

4 User1 9

4 User1 8

4 User3 7

-------------

На выходе получаем:

w user cnt sum

1 User1 0 0

1 User2 7 7

1 User3 2 2

2 User1 4 4

2 User2 2 9

2 User3 0 2

3 User1 0 4

3 User2 9 18

3 User3 6 8

4 User1 17 21

4 User2 0 18

4 User3 7 15


In [11]:
#Так как ранее мы импортировали pd, сейчас этот делать не будем

data = {
    'w': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4],
    'user': ['User2', 'User3', 'User1', 'User1', 'User2', 'User2', 'User2', 'User3', 'User1', 'User1', 'User3'],
    'c': [7, 2, 3, 1, 2, 5, 4, 6, 9, 8, 7]
}

df = pd.DataFrame(data)
grouped = df.groupby(['w', 'user'])['c'].sum().reset_index()
index = pd.MultiIndex.from_product([df['w'].unique(), df['user'].unique()], names=['w', 'user'])

In [13]:
grouped_full = grouped.set_index(['w', 'user']).reindex(index, fill_value=0).reset_index()
grouped_full['sum'] = grouped_full.groupby('user')['c'].cumsum()
grouped_full.columns = ['w', 'user', 'cnt', 'sum']
grouped_full.sort_values(by=['w', 'user'], inplace=True)

print(grouped_full.to_string(index=False))

 w  user  cnt  sum
 1 User1    0    0
 1 User2    7    7
 1 User3    2    2
 2 User1    4    4
 2 User2    2    9
 2 User3    0    2
 3 User1    0    4
 3 User2    9   18
 3 User3    6    8
 4 User1   17   21
 4 User2    0   18
 4 User3    7   15
